In [1]:
import pandas as pd
import numpy as np
import json
import numpy as np
import os
import networkx as nx
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

from spektral.layers import GraphConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

from collections import Counter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
genres_songs_map=open("genres_vs_songs.json")
genres_songs_map=json.load(genres_songs_map)
for i in genres_songs_map:
    print(i+"-- %d"%(len(genres_songs_map[i])))

Classical-- 1000
Electronic-- 1000
Experimental-- 1000
Folk-- 1000
Hip-Hop-- 1000
Instrumental-- 1000
International-- 1000
Jazz-- 1000
Pop-- 1000
Rock-- 1000


In [3]:
song_index_map=dict()
song_genres_map=dict()
index_song_map=dict()
for i in genres_songs_map.values():
    for j in i:
        song_genres_map[j]=list()
for i in genres_songs_map:
    for j in genres_songs_map[i]:
        song_genres_map[j].append(i)
aux=0
for i in song_genres_map:
    song_index_map[i]=aux
    aux=aux+1
for i in song_index_map:
    index_song_map[song_index_map[i]]=i

one_gener_song=list()
more_than_one_gener_song=list()
count_one=0
count_two=0
count_more_than_two=0
for i in song_genres_map.values():
    if(len(i)==1):count_one=count_one+1
    elif(len(i)==2): count_two=count_two+1
    else: count_more_than_two=count_more_than_two+1
for i in song_genres_map:
    if(len(song_genres_map[i])==1):one_gener_song.append(i)
    else: more_than_one_gener_song.append(i)
print(count_one)
print(count_two)
print(count_more_than_two)

9425
283
3


In [4]:
temp=dict()
test=dict()
validation=dict()
train=dict()
for i in genres_songs_map.keys():
    temp[i]=list()
    test[i]=list()
    validation[i]=list()
    train[i]=list()
for i in one_gener_song:
    temp[song_genres_map[i][0]].append(i)

import random
from random import sample 

random.seed(1)
for i in temp:
    aux=sample(temp[i],int(len(temp[i])*0.3))
    test[i]=aux[0:int(len(aux)/2)]
    validation[i]=aux[int(len(aux)/2):]
    for j in temp[i]:
        if(j not in aux): train[i].append(j)
for i in more_than_one_gener_song:
    for j in song_genres_map[i]:
        train[j].append(i)

In [5]:
# 0.3 from each genre
for i in genres_songs_map.keys():
    print(i+"-- Train: % 2d, Validation: % 2d, Test: % 2d"%(len(train[i]), len(validation[i]), len(test[i])))

Classical-- Train:  717, Validation:  142, Test:  141
Electronic-- Train:  717, Validation:  142, Test:  141
Experimental-- Train:  719, Validation:  141, Test:  140
Folk-- Train:  721, Validation:  140, Test:  139
Hip-Hop-- Train:  711, Validation:  145, Test:  144
Instrumental-- Train:  718, Validation:  141, Test:  141
International-- Train:  719, Validation:  141, Test:  140
Jazz-- Train:  723, Validation:  139, Test:  138
Pop-- Train:  718, Validation:  141, Test:  141
Rock-- Train:  713, Validation:  144, Test:  143


In [6]:
test_song_id=list()
for i in test.values():
    for j in i:
        test_song_id.append(j)
train_song_id=list()
for i in train.values():
    for j in i:
        train_song_id.append(j)
validation_song_id=list()
for i in validation.values():
    for j in i:
        validation_song_id.append(j)

In [7]:
classes=[None for _ in range(10)]
temp_index=dict()
count=0
for i in genres_songs_map.keys():
    temp_index[i]=count
    count=count+1
for i in temp_index: classes[temp_index[i]]=i
    
Y=np.zeros((max(index_song_map.keys())+1, 10))
for i in index_song_map:
    for j in song_genres_map[index_song_map[i]]:
        Y[i][temp_index[j]]=1/len(song_genres_map[index_song_map[i]])

train_bool=np.zeros((max(index_song_map.keys())+1,),dtype=bool)
validation_bool=np.zeros((max(index_song_map.keys())+1,),dtype=bool)
test_bool=np.zeros((max(index_song_map.keys())+1,),dtype=bool)

for i in index_song_map:
    if(index_song_map[i] in train_song_id):train_bool[i]=True
    elif(index_song_map[i] in validation_song_id):validation_bool[i]=True
    else: test_bool[i]=True

In [8]:
features=pd.read_csv('features.csv')
features=features.loc[3:]
temp=dict()
for i in features.to_numpy():
    if(int(i[0]) in song_genres_map.keys()):temp[int(i[0])]=[float(item) for item in i[1:].tolist()]

X=list()
for i in range(max(index_song_map.keys())+1):
    X.append(temp[index_song_map[i]])
X=np.asarray(X)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
X = StandardScaler().fit_transform(X)
pca=PCA(n_components=187)
X=pca.fit_transform(X)

e:\python\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,

In [10]:
import numpy as np 
from scipy.sparse import csr_matrix
A=csr_matrix((max(song_index_map.values())+1, max(song_index_map.values())+1),dtype = np.int8).toarray() 
for i in train:
    for j in train[i]:
        for k in train[i]:
            if(j!=k): 
                A[song_index_map[j]][song_index_map[k]]=1
                A[song_index_map[k]][song_index_map[j]]=1

from scipy import spatial

for i in validation_song_id:
    for j in train_song_id:
        temp_val=1-spatial.distance.cosine(X[song_index_map[i]], X[song_index_map[j]])
        if(temp_val>=0.5):
            A[song_index_map[i]][song_index_map[j]]=1
            A[song_index_map[j]][song_index_map[i]]=1
#             print(str(song_genres_map[i])+" "+str(song_genres_map[j])+" "+str(temp_val))

for i in test_song_id:
    for j in train_song_id:
        temp_val=1-spatial.distance.cosine(X[song_index_map[i]], X[song_index_map[j]])
        if(temp_val>=0.5):
            A[song_index_map[i]][song_index_map[j]]=1
            A[song_index_map[j]][song_index_map[i]]=1

In [11]:
import networkx as nx
A[A > 0] = 1
G = nx.Graph(A)
A = nx.adjacency_matrix(G)
print('Graph info: ', nx.info(G))

Graph info:  Name: 
Type: Graph
Number of nodes: 9711
Number of edges: 2714045
Average degree: 558.9630


In [28]:
# Parameters
channel_1 = 100
channel_2 = 20# Number of channels in the first layer
dropout = 0.9          # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 2000           # Number of training epochs
es_patience = 10        # Patience for early stopping

# Preprocessing operations
A = GraphConv.preprocess(A).astype('f4')

# Model definition
X_in = Input(shape=(X.shape[1], ))
fltr_in = Input((X.shape[0], ), sparse=True)

graph_conv_1 = GraphConv(channel_1,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([X_in,fltr_in])

dropout_1 = Dropout(dropout)(graph_conv_1)

# graph_conv_2 = GraphConv(channel_2,
#                          activation='relu',
#                          kernel_regularizer=l2(l2_reg),
#                          use_bias=False)([dropout_1, fltr_in])
# dropout_2 = Dropout(dropout)(graph_conv_2)


graph_conv_3 = GraphConv(10,
                         activation='softmax',
                         use_bias=False)([dropout_1, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_3)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./Tensorboard_GCN_FMA',
)
callback_GCN = [tbCallBack_GCN] 

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 187)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 9711)]       0                                            
__________________________________________________________________________________________________
graph_conv_4 (GraphConv)        (None, 100)          18700       input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 100)          0           graph_conv_4[0][0]    

In [29]:
validation_data = ([X, A], Y, validation_bool)
model.fit([X, A],
          Y,
          sample_weight=train_bool,
          epochs=epochs,
          batch_size=X.shape[0],
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
              EarlyStopping(patience=es_patience,  restore_best_weights=True),
              tbCallBack_GCN
          ])


Epoch 1/2000
1/1 [==============================] - 1s 1s/step - loss: 3.3265 - acc: 0.1041 - val_loss: 0.4682 - val_acc: 0.1448
Epoch 2/2000
1/1 [==============================] - 1s 507ms/step - loss: 2.8808 - acc: 0.1301 - val_loss: 0.4188 - val_acc: 0.2076
Epoch 3/2000
1/1 [==============================] - 1s 504ms/step - loss: 2.5496 - acc: 0.1546 - val_loss: 0.3913 - val_acc: 0.2535
Epoch 4/2000
1/1 [==============================] - 0s 495ms/step - loss: 2.2223 - acc: 0.1931 - val_loss: 0.3760 - val_acc: 0.2733
Epoch 5/2000
1/1 [==============================] - 1s 528ms/step - loss: 2.0829 - acc: 0.2197 - val_loss: 0.3666 - val_acc: 0.2895
Epoch 6/2000
1/1 [==============================] - 0s 497ms/step - loss: 1.8676 - acc: 0.2660 - val_loss: 0.3604 - val_acc: 0.2980
Epoch 7/2000
1/1 [==============================] - 0s 499ms/step - loss: 1.7245 - acc: 0.2791 - val_loss: 0.3556 - val_acc: 0.3150
Epoch 8/2000
1/1 [==============================] - 1s 513ms/step - loss: 1.618

In [30]:
X_te = X[test_bool]
A_te = A[test_bool,:][:,test_bool]
y_te = Y[test_bool]

y_pred = model.predict([X_te, A_te], batch_size=X.shape[0])

from sklearn.metrics import classification_report
report = classification_report(np.argmax(y_te,axis=1), np.argmax(y_pred,axis=1), target_names=classes)
print('GCN Classification Report: \n {}'.format(report))

GCN Classification Report: 
                precision    recall  f1-score   support

    Classical       0.52      0.55      0.53       141
   Electronic       0.26      0.33      0.29       141
 Experimental       0.23      0.16      0.19       140
         Folk       0.32      0.35      0.33       139
      Hip-Hop       0.42      0.53      0.46       144
 Instrumental       0.26      0.22      0.24       141
International       0.39      0.29      0.33       140
         Jazz       0.37      0.41      0.39       138
          Pop       0.25      0.15      0.19       141
         Rock       0.37      0.49      0.42       143

     accuracy                           0.35      1408
    macro avg       0.34      0.35      0.34      1408
 weighted avg       0.34      0.35      0.34      1408



In [15]:
# # 0.2 from each genre
# for i in genres_songs_map.keys():
#     print(i+"-- Train: % 2d, Validation: % 2d, Test: % 2d"%(len(train[i]), len(validation[i]), len(test[i])))

In [16]:
# import json

In [17]:
# def set_default(obj):
#     if isinstance(obj, set):return list(obj)
#     raise TypeError
# file=json.dumps(train, sort_keys=True, indent=1, default=set_default)
# f = open('train.json', 'w')
# f.write(file)
# f.close()

In [18]:
# def set_default(obj):
#     if isinstance(obj, set):return list(obj)
#     raise TypeError
# file=json.dumps(test, sort_keys=True, indent=1, default=set_default)
# f = open('test.json', 'w')
# f.write(file)
# f.close()

In [19]:
# def set_default(obj):
#     if isinstance(obj, set):return list(obj)
#     raise TypeError
# file=json.dumps(validation, sort_keys=True, indent=1, default=set_default)
# f = open('validation.json', 'w')
# f.write(file)
# f.close()

In [20]:
# def set_default(obj):
#     if isinstance(obj, set):return list(obj)
#     raise TypeError
# file=json.dumps(song_index_map, sort_keys=True, indent=1, default=set_default)
# f = open('song_index_map.json', 'w')
# f.write(file)
# f.close()

In [21]:

# pca = PCA(n_components=X.shape[1])
# pca.fit(X)
# #print (pca.explained_variance_ratio_)
# #print (pca.explained_variance_ratio_.cumsum())
# pcount=0
# for i in pca.explained_variance_ratio_.cumsum():
#     pcount=pcount+1
#     if(i>=.95):
#         print(pcount)
#         break

In [22]:
# from sklearn.decomposition import KernelPCA
# kpca=KernelPCA(n_components=X.shape[1])
# kpca_transform=kpca.fit_transform(X)

In [23]:
# explained_variance = np.var(kpca_transform, axis=0) 
# explained_variance_ratio = explained_variance / np.sum(explained_variance) 
# explained_variance_ratio
# kcount=0
# for i in explained_variance_ratio.cumsum():
#     kcount=kcount+1
#     if(i>=.98):
#         print(kcount)
#         break

In [24]:
# kpca=KernelPCA(n_components=kcount)
# X=kpca.fit_transform(X)
